# Operating on Data in Pandas

One of the essential pieces of NumPy is the ability to perform quick element-wise operations, both with basic arithmetic (addition, subtraction, multiplication, etc.) and with more sophisticated operations (trigonometric functions, exponential and logarithmic functions, etc.).
Pandas inherits much of this functionality from NumPy, and the ufuncs that we introduced in [Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb) are key to this.

Pandas includes a couple useful twists, however: for unary operations like negation and trigonometric functions, these ufuncs will *preserve index and column labels* in the output, and for binary operations such as addition and multiplication, Pandas will automatically *align indices* when passing the objects to the ufunc.
This means that keeping the context of data and combining data from different sources–both potentially error-prone tasks with raw NumPy arrays–become essentially foolproof ones with Pandas.
We will additionally see that there are well-defined operations between one-dimensional ``Series`` structures and two-dimensional ``DataFrame`` structures.

## Ufuncs: Index Preservation

Because Pandas is designed to work with NumPy, any NumPy ufunc will work on Pandas ``Series`` and ``DataFrame`` objects.
Let's start by defining a simple ``Series`` and ``DataFrame`` on which to demonstrate this:

In [1]:
import pandas as pd
import numpy as np

In [2]:
rng = np.random.RandomState()
ser = pd.Series(rng.random(10))
print(ser)

0    0.663423
1    0.629918
2    0.093337
3    0.526354
4    0.479459
5    0.353919
6    0.993170
7    0.293500
8    0.537729
9    0.280019
dtype: float64


In [3]:
print(id(rng))

1851946401864


In [4]:
type(ser)

pandas.core.series.Series

In [5]:
df = pd.DataFrame(rng.randint(0, 10, (4, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,0,6,0,9
1,1,9,3,1
2,8,7,0,7
3,8,6,5,9


In [6]:
df.mean(axis = 0) #Row

A    4.25
B    7.00
C    2.00
D    6.50
dtype: float64

In [7]:
df.mean(1)   # axis =1 Col

0    3.75
1    3.50
2    5.50
3    7.00
dtype: float64

In [9]:
df.median(axis = 'rows')

A    4.5
B    6.5
C    1.5
D    8.0
dtype: float64

If we apply a NumPy ufunc on either of these objects, the result will be another Pandas object *with the indices preserved:*

In [10]:
np.exp(ser)

0    1.941427
1    1.877456
2    1.097832
3    1.692749
4    1.615201
5    1.424640
6    2.699780
7    1.341113
8    1.712115
9    1.323155
dtype: float64

Or, for a slightly more complex calculation:

In [11]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,0.000000e+00,-1.000000,0.000000,0.707107
1,7.071068e-01,0.707107,0.707107,0.707107
2,-2.449294e-16,-0.707107,0.000000,-0.707107
3,-2.449294e-16,-1.000000,-0.707107,0.707107


Any of the ufuncs discussed in [Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb) can be used in a similar manner.

## UFuncs: Index Alignment

For binary operations on two ``Series`` or ``DataFrame`` objects, Pandas will align indices in the process of performing the operation.
This is very convenient when working with incomplete data, as we'll see in some of the examples that follow.

### Index alignment in Series

As an example, suppose we are combining two different data sources, and find only the top three US states by *area* and the top three US states by *population*:

In [12]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population')
area

Alaska        1723337
Texas          695662
California     423967
Name: area, dtype: int64

In [13]:
print(population)

California    38332521
Texas         26448193
New York      19651127
Name: population, dtype: int64


Let's see what happens when we divide these to compute the population density:

In [14]:
population / area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

The resulting array contains the *union* of indices of the two input arrays, which could be determined using standard Python set arithmetic on these indices:

In [15]:
area.index | population.index

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

Any item for which one or the other does not have an entry is marked with ``NaN``, or "Not a Number," which is how Pandas marks missing data (see further discussion of missing data in [Handling Missing Data](03.04-Missing-Values.ipynb)).
This index matching is implemented this way for any of Python's built-in arithmetic expressions; any missing values are filled in with NaN by default:

In [16]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A

0    2
1    4
2    6
dtype: int64

In [17]:
B

1    1
2    3
3    5
dtype: int64

In [18]:
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

In [19]:
A*B

0     NaN
1     4.0
2    18.0
3     NaN
dtype: float64

If using NaN values is not the desired behavior, the fill value can be modified using appropriate object methods in place of the operators.
For example, calling ``A.add(B)`` is equivalent to calling ``A + B``, but allows optional explicit specification of the fill value for any elements in ``A`` or ``B`` that might be missing:

In [20]:
print(A)
print(B)

0    2
1    4
2    6
dtype: int64
1    1
2    3
3    5
dtype: int64


In [26]:
fill = np.mean(A).astype(int)
fill

4

In [24]:
A.add(B, fill_value=fill)

0    6.0
1    5.0
2    9.0
3    9.0
dtype: float64

### Index alignment in DataFrame

A similar type of alignment takes place for *both* columns and indices when performing operations on ``DataFrame``s:

In [27]:
A = pd.DataFrame(rng.randint(0, 20, (3, 5)),
                 columns=list('ABCDE'))
A

,A,B,C,D,E
0,14,19,0,10,3
1,17,14,10,11,7
2,8,11,2,16,11


In [28]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),
                 columns=list('BAC'))
B

,B,A,C
0,1,2,5
1,3,6,8
2,0,8,8


In [30]:
A + B

,A,B,C,D,E
0,16,20,5,NaN,NaN
1,23,17,18,NaN,NaN
2,16,11,10,NaN,NaN


In [31]:
A*B

,A,B,C,D,E
0,28,19,0,NaN,NaN
1,102,42,80,NaN,NaN
2,64,0,16,NaN,NaN


Notice that indices are aligned correctly irrespective of their order in the two objects, and indices in the result are sorted.
As was the case with ``Series``, we can use the associated object's arithmetic method and pass any desired ``fill_value`` to be used in place of missing entries.
Here we'll fill with the mean of all values in ``A`` (computed by first stacking the rows of ``A``):

In [32]:
Data = A.stack()
Data

0  A    14
   B    19
   C     0
   D    10
   E     3
1  A    17
   B    14
   C    10
   D    11
   E     7
2  A     8
   B    11
   C     2
   D    16
   E    11
dtype: int32

In [33]:
Data.sum()/len(Data)

10.2

In [34]:
fill = A.stack().mean()
fill

10.2

In [35]:
A.add(B, fill_value=fill,axis=0)

,A,B,C,D,E
0,16,20,5,20.2,13.2
1,23,17,18,21.2,17.2
2,16,11,10,26.2,21.2


In [36]:
A.add(B, fill_value=fill,axis='rows')

,A,B,C,D,E
0,16,20,5,20.2,13.2
1,23,17,18,21.2,17.2
2,16,11,10,26.2,21.2


The following table lists Python operators and their equivalent Pandas object methods:

| Python Operator | Pandas Method(s)                      |
|-----------------|---------------------------------------|
| ``+``           | ``add()``                             |
| ``-``           | ``sub()``, ``subtract()``             |
| ``*``           | ``mul()``, ``multiply()``             |
| ``/``           | ``truediv()``, ``div()``, ``divide()``|
| ``//``          | ``floordiv()``                        |
| ``%``           | ``mod()``                             |
| ``**``          | ``pow()``                             |


## Ufuncs: Operations Between DataFrame and Series

When performing operations between a ``DataFrame`` and a ``Series``, the index and column alignment is similarly maintained.
Operations between a ``DataFrame`` and a ``Series`` are similar to operations between a two-dimensional and one-dimensional NumPy array.
Consider one common operation, where we find the difference of a two-dimensional array and one of its rows:

In [37]:
A = rng.randint(10, size=(3, 4))
A

array([[0, 1, 4, 0],
       [0, 7, 3, 2],
       [8, 4, 1, 1]])

In [38]:
A[0]

array([0, 1, 4, 0])

In [39]:
A - A[0]

array([[ 0,  0,  0,  0],
       [ 0,  6, -1,  2],
       [ 8,  3, -3,  1]])

According to NumPy's broadcasting rules (see [Computation on Arrays: Broadcasting](02.05-Computation-on-arrays-broadcasting.ipynb)), subtraction between a two-dimensional array and one of its rows is applied row-wise.

In Pandas, the convention similarly operates row-wise by default:

In [40]:
df = pd.DataFrame(A, columns=list('QRST'))
df

,Q,R,S,T
0,0,1,4,0
1,0,7,3,2
2,8,4,1,1


In [41]:
df['Q'][0]

0

In [42]:
df.iloc[0] # Integer loc or index loc which used to select perticular Row

Q    0
R    1
S    4
T    0
Name: 0, dtype: int32

In [43]:
df - df.iloc[0]  

,Q,R,S,T
0,0,0,0,0
1,0,6,-1,2
2,8,3,-3,1


If you would instead like to operate column-wise, you can use the object methods mentioned earlier, while specifying the ``axis`` keyword:

In [44]:
df['R']

0    1
1    7
2    4
Name: R, dtype: int32

In [45]:
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,-1,0,3,-1
1,-7,0,-4,-5
2,4,0,-3,-3


In [46]:
df.subtract(df['R'], axis=1)

,Q,R,S,T,0,1,2
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Note that these ``DataFrame``/``Series`` operations, like the operations discussed above, will automatically align  indices between the two elements:

In [47]:
df

,Q,R,S,T
0,0,1,4,0
1,0,7,3,2
2,8,4,1,1


In [48]:
halfrow = df.iloc[1, ::2]
halfrow

Q    0
S    3
Name: 1, dtype: int32

In [49]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,1.0,NaN
1,0.0,NaN,0.0,NaN
2,8.0,NaN,-2.0,NaN


This preservation and alignment of indices and columns means that operations on data in Pandas will always maintain the data context, which prevents the types of silly errors that might come up when working with heterogeneous and/or misaligned data in raw NumPy arrays.